In [1]:
import pandas as pd


In [9]:
df1 = pd.read_csv('sam_csvs\sam_test1.csv')
df1.columns

df = df1[['POSITION X', 'POSITION Y', 'CD68', 'MDSC']]
print(df.head)

<bound method NDFrame.head of         POSITION X  POSITION Y  CD68  MDSC
0        51257.254   42903.758     0     0
1        51619.285   42545.438     1     0
2        51489.516   42677.035     0     0
3        51618.191   42549.020     0     1
4        51434.113   42733.695     1     0
...            ...         ...   ...   ...
115110   48725.254   40357.273     0     0
115111   48741.340   40339.059     0     0
115112   48718.805   40368.750     0     0
115113   48733.887   40358.613     0     0
115114   48724.695   40379.027     0     0

[115115 rows x 4 columns]>


<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Maltez Lab\AppData\Local\Temp\ipykernel_13948\428672312.py:1: SyntaxWarning: invalid escape sequence '\s'
  df1 = pd.read_csv('sam_csvs\sam_test1.csv')


In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree

# Extract position arrays for each group
cd68_positions = df[df['CD68'] == 1][['POSITION X', 'POSITION Y']].values
mdsc_positions = df[df['MDSC'] == 1][['POSITION X', 'POSITION Y']].values

# Build KDTree for MDSC positions
mdsc_tree = cKDTree(mdsc_positions)

# Compute distances for CD68-positive cells
distances = []
cd68_indices = df[df['CD68'] == 1].index

for idx, (x, y) in zip(cd68_indices, cd68_positions):
    if df.loc[idx, 'MDSC'] == 1:
        distances.append(0.0)
    else:
        dist, _ = mdsc_tree.query([x, y])
        distances.append(dist)

# Assign result back to a new column in the original DataFrame
df.loc[cd68_indices, 'DIST_TO_CLOSEST_MDSC'] = distances

# Optional: fill NaNs with 0 (or leave them if you want to distinguish non-CD68 rows)
df['DIST_TO_CLOSEST_MDSC'] = df['DIST_TO_CLOSEST_MDSC'].fillna(0)

# 🔍 Final Filter: keep only rows where CD68 == 1 or MDSC == 1
filtered_df = df[(df['CD68'] == 1) | (df['MDSC'] == 1)]

# Preview
filtered_df #[['POSITION X', 'POSITION Y', 'CD68', 'MDSC', 'DIST_TO_CLOSEST_MDSC']]

C:\Users\Maltez Lab\AppData\Local\Temp\ipykernel_13948\251496218.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DIST_TO_CLOSEST_MDSC'] = df['DIST_TO_CLOSEST_MDSC'].fillna(0)


,POSITION X,POSITION Y,CD68,MDSC,DIST_TO_CLOSEST_MDSC
1,51619.285,42545.438,1,0,3.745338
3,51618.191,42549.020,0,1,0.000000
4,51434.113,42733.695,1,0,35.942790
5,51478.602,42688.203,1,0,68.813118
6,51325.926,42841.258,1,0,8.287765
...,...,...,...,...,...
115047,48661.957,40872.906,1,0,10.109420
115051,49046.715,40492.348,1,0,60.086654
115061,49046.445,40486.332,1,0,58.517117
115076,49263.344,40169.195,1,0,64.095221
